# Постановка задачи

Прочитать про **word embeddings** и **bag of words** и классифицировать отзывы на фильмы из рейтинга **IMDB** (встроенный в keras датасет).

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.datasets import imdb # импортируем из модуля keras.datasets класс imdb (база данных фильмов из рейтинга IMDB)

In [80]:
num_words = 2000 # размер словаря (количество значений для кодирования слов)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words) # импортируем тренируемые и тестовые данные из датасета imdb

In [81]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 1920, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 2, 2, 134, 26, 4, 715, 8, 118, 1634

## Предобработка данных

In [82]:
max_review_length = 250 # по этому количеству слов сокращается отзыв
x_train = pad_sequences(x_train, maxlen=max_review_length) # используем метод pad_sequences, чтобы последовательности в списках имели одинаковую длину; этот метод заполняет нулями (0) начало каждой последовательности до тех пор, пока каждая последовательность не будет иметь ту же длину, что и самая длинная последовательность 
x_test = pad_sequences(x_test, maxlen=max_review_length)

In [83]:
x_train

array([[  0,   0,   0, ...,  19, 178,  32],
       [  0,   0,   0, ...,  16, 145,  95],
       [  0,   0,   0, ...,   7, 129, 113],
       ...,
       [  0,   0,   0, ...,   4,   2,   2],
       [  0,   0,   0, ...,  12,   9,  23],
       [  0,   0,   0, ..., 204, 131,   9]], dtype=int32)

In [86]:
# приводим значения массивов к вещественным числам
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [87]:
x_train

array([[  0.,   0.,   0., ...,  19., 178.,  32.],
       [  0.,   0.,   0., ...,  16., 145.,  95.],
       [  0.,   0.,   0., ...,   7., 129., 113.],
       ...,
       [  0.,   0.,   0., ...,   4.,   2.,   2.],
       [  0.,   0.,   0., ...,  12.,   9.,  23.],
       [  0.,   0.,   0., ..., 204., 131.,   9.]], dtype=float32)

In [88]:
scaler = MinMaxScaler(feature_range=(0, 1)) # сжимаем данные от 0 до 1 (нормализируем данные), чтобы нейросети было проще обрабатывать данные
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [89]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.00852558, 0.08817635,
        0.01503006],
       [0.        , 0.        , 0.        , ..., 0.00702106, 0.07164329,
        0.04659319],
       [0.        , 0.        , 0.        , ..., 0.00250752, 0.06362725,
        0.05561122],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00100301, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00501505, 0.00350701,
        0.01052104],
       [0.        , 0.        , 0.        , ..., 0.10130391, 0.06462926,
        0.00350701]], dtype=float32)

In [90]:
x_train.shape

(25000, 250)

In [91]:
x_test

array([[0.        , 0.        , 0.        , ..., 0.00601202, 0.002003  ,
        0.35821643],
       [0.00100908, 0.01155198, 0.65631264, ..., 0.06162325, 0.0010015 ,
        0.        ],
       [0.35872856, 0.00100452, 0.001002  , ..., 0.00350701, 0.02754131,
        0.48747495],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00951904, 0.42263395,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00250376,
        0.23446895],
       [0.        , 0.        , 0.        , ..., 0.01603206, 0.        ,
        0.        ]], dtype=float32)

In [92]:
x_test.shape

(25000, 250)

## Создание модели

In [110]:
embedding_vector_length = 32 # размер векторного пространства, в которое будут встроены слова (n-мерный вектор, в который будет преобразовано каждое слово) - https://yandex.ru/q/tech/10818735873/
model = Sequential()
model.add(Embedding(input_dim=32, output_dim=embedding_vector_length, input_length=max_review_length)) # задаём слой внедрения; input_dim - количество отдельных слов в обучающем наборе (размер словаря); output_dim - размер векторов вложения; input_length - длина входных последовательностей (длина отзыва)
model.add(LSTM(32))
model.add(Dense(1))

In [111]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 250, 32)           1024      
                                                                 
 lstm_11 (LSTM)              (None, 32)                8320      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 9,377
Trainable params: 9,377
Non-trainable params: 0
_________________________________________________________________


In [112]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) # компилируем модель

In [113]:
model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 48s 72ms/step - loss: 0.2545 - accuracy: 0.4986 - val_loss: 0.2521 - val_accuracy: 0.4942
Epoch 2/10
625/625 [==============================] - 44s 71ms/step - loss: 0.2509 - accuracy: 0.5007 - val_loss: 0.2504 - val_accuracy: 0.4942
Epoch 3/10
625/625 [==============================] - 46s 74ms/step - loss: 0.2505 - accuracy: 0.5068 - val_loss: 0.2499 - val_accuracy: 0.5062
Epoch 4/10
625/625 [==============================] - 44s 71ms/step - loss: 0.2506 - accuracy: 0.5044 - val_loss: 0.2516 - val_accuracy: 0.4942
Epoch 5/10
625/625 [==============================] - 46s 74ms/step - loss: 0.2507 - accuracy: 0.4996 - val_loss: 0.2499 - val_accuracy: 0.5062
Epoch 6/10
625/625 [==============================] - 56s 90ms/step - loss: 0.2504 - accuracy: 0.5057 - val_loss: 0.2501 - val_accuracy: 0.5062
Epoch 7/10
625/625 [==============================] - 48s 77ms/step - loss: 0.2505 - accuracy: 0.4999 - val_loss: 0.2515 - val_accuracy: